In [1]:
import sys
import os

import sqlite3 as db
import sqlalchemy as sa
import logging as logging

import datetime as dt
import time

In [2]:
# "global" variables
wc_log = None
startup_path = os.getcwd()
db_dir = None
db_engine = None

In [3]:
def _init_logs(reset_handlers=True):
    """Create a sys.stdout logger for w_c and print startup information 
    
    The logger name is the "w_c" namespace
    """
    global wc_log
    log = logging.getLogger("w_c")
    if reset_handlers:
        log.handlers = []
        log_stream = logging.StreamHandler(sys.stdout)
        log_formatter = logging.Formatter('%(asctime)s::%(name)s::%(levelname)s::%(message)s')
        log_stream.setFormatter(log_formatter)
        log.addHandler(log_stream)
        log.setLevel(logging.DEBUG)
    log.debug("Starting in %s" % startup_path)
    log.debug("Versions:")
    log.debug("  slqalchemy: {}".format(sa.__version__))
    log.debug("     sqlite3: {}".format(db.version))
    wc_log = log

_init_logs()

2016-05-30 09:43:38,766::w_c::DEBUG::Starting in C:\Users\ibuypower\git\w.c
2016-05-30 09:43:38,769::w_c::DEBUG::Versions:
2016-05-30 09:43:38,772::w_c::DEBUG::  slqalchemy: 1.0.9
2016-05-30 09:43:38,776::w_c::DEBUG::     sqlite3: 2.6.0


In [9]:
def _setup_engine():
    '''Set the database executioin file and create a db directory if none exists'''
    global db_dir, db_engine 
    db_dir = os.path.join(os.path.dirname(startup_path),'db')
    if not os.path.exists(db_dir):
        os.mkdir(db_dir)
    db_name = os.path.join(db_dir, "work_cards.db")
    wc_log.debug(" db_dir:: %s" % db_dir)
    wc_log.debug("db_name:: %s" % db_name)
    db_engine = sa.create_engine("sqlite:///" + db_name )
    
_setup_engine()

TypeError: exists() takes exactly 1 argument (0 given)

In [5]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [6]:

class Tasks(Base):
    '''SQLAlchemy Tasks class that maps all tasks to the tasks table
    
    Task descriptions default to Twitter length strings :-)
    '''
    __tablename__ = "tasks"
    id = sa.Column(sa.Integer, primary_key = True)
    desc = sa.Column(sa.String(140))
    orig_comp = sa.Column(sa.DateTime)
    cur_comp = sa.Column(sa.DateTime)
    for_whom = sa.Column(sa.Integer)
    
    def __repr__(self):
        return "Task {}: ({}, {}, {}, {})".format(
        self.id, self.desc, self.orig_comp, self.cur_comp, self.for_whom )
    

In [7]:
Tasks.__table__

Table('tasks', MetaData(bind=None), Column('id', Integer(), table=<tasks>, primary_key=True, nullable=False), Column('desc', String(length=140), table=<tasks>), Column('orig_comp', DateTime(), table=<tasks>), Column('cur_comp', DateTime(), table=<tasks>), Column('for_whom', Integer(), table=<tasks>), schema=None)

In [8]:
Base.metadata.create_all(db_engine)

OperationalError: (sqlite3.OperationalError) unable to open database file

In [ ]:
first_task = Tasks(desc = "Buy weekly groceries", orig_comp=dt.datetime(2016, 5, 29),
                  cur_comp=dt.datetime(2016, 6, 4), for_whom=1)

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db_engine)

In [ ]:
session = Session()

In [ ]:
session.add(first_task)

In [ ]:
session.flush()

In [ ]:
session.commit()

In [ ]:
for task in session.query(Tasks).filter_by():
    print(task)